In [2]:
!nvidia-smi 

Sat Mar 25 00:22:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    44W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 93.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 102.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 25.3 MB/s eta 0:00:00


In [ ]:
%%sh
mkdir machine-learning
rsync -av --progress /content/drive/MyDrive//machine-learning/* /content/machine-learning/ --exclude model-output/model_files/
# cp -r /content/drive/MyDrive//machine-learning/* /content/machine-learning/
# cp /content/drive/MyDrive/Arbor/machine-learning/brand/model-preparation/News_processed_031_060.csv /content/machine-learning/model-preparation/

In [29]:
DATE = '2023_03_24_'
MODEL_RUN = 1
PREDICTION_RUN = '241950' 

In [13]:

train_file_name    = '05.Senti4SD_GoldStandard_EmotionPolarity-ML.csv'
additional_testing_file_name    = 'library-selection-factors.csv'
run_additional_testing = False

#train_file_name    = "BenchmarkUddinSO-ConsoliatedAspectSentiment-ML.csv"
#train_file_name    = "BenchmarkUddinSO-ConsoliatedAspectSentiment-ML-D.csv"
#train_file_name    = "BenchmarkUddinSO-ConsoliatedAspectSentiment-ML-O.csv"
#train_file_name    = "07.SO.BERT4SentiSE-ML.csv"
#train_file_name    = "07.SO.BERT4SentiSE.Combined-ML.csv"



topic_title = "sentiment"
logging_steps = 50
num_train_epochs = 2

data_dir   = "/content/machine-learning/model-preparation/"
output_dir = "/content/machine-learning/model-output/"

# for multiple model and dataset files
dataset_dir   = "/content/machine-learning/model-preparation/0.Combined-Dataset/"
model_output_dir_base = "/content/machine-learning/model-output/model_files/"
prediction_output_dir = "/content/machine-learning/model-output/prediction_files/"

train_file_path = data_dir+train_file_name

#predict_file_name = "StackOverflow_Test.csv"
#predict_file_path = data_dir+predict_file_name


In [30]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, RobertaForSequenceClassification, RobertaTokenizer
#          Model          | Tokenizer          | Pretrained weights shortcut
# MODELS = [(AutoModelForSequenceClassification,AutoTokenizer,'princeton-nlp/sup-simcse-roberta-large'),
#           (RobertaForSequenceClassification, RobertaTokenizer,'roberta-large'), 
#          ]

MODELS = {
    "roberta":{
        'classifier': RobertaForSequenceClassification,
        'tokenizer': RobertaTokenizer,
        'path': 'roberta-large',
        'logging_steps': 100,
        'num_train_epochs': 4,
        'max_length': 512,
        'learning_rate': 2e-5,
        'early_stopping_patience': 3,
    },
    "simcse":{
        'classifier': AutoModelForSequenceClassification,
        'tokenizer': AutoTokenizer,
        'path': 'princeton-nlp/sup-simcse-roberta-large',
        'logging_steps': 100,
        'num_train_epochs': 4,
        'max_length': 512,
        'learning_rate': 2e-5,
        'early_stopping_patience': 3000,
    }
}

In [15]:

import numpy as np

def read_clean_data(file_path):
  data = pd.read_csv(file_path)
  nan_value = float("NaN")
  #Convert NaN values to empty string
  data["text"].replace("", nan_value, inplace=True)
  data["text"].replace([np.inf, -np.inf], nan_value, inplace=True)
  data.dropna(subset = ["text"], inplace=True)

  return data

def get_ML_data(type, file_path = ''):
  # if from_file:
  #   if type == "Train":
  #     return read_clean_data(train_file_path)

  #   if type == "Validate":
  #     return read_clean_data(validate_file_path)

  #   if type == "Test":
  #     return read_clean_data(predict_file_path)

  if len(file_path) == 0:
    file_path = train_file_path

  df_ml = read_clean_data(file_path)

  # get three dataframes, one for each set
  return df_ml[df_ml['MLSet'] == type]
  
      
def print_title(model, SEP, dataset, dataset_type):
  combination_title = (model+SEP+dataset+SEP+dataset_type).replace('/', '-')
  print('*'*80)
  print(combination_title.upper())
  print('*'*80)    


In [16]:
#https://towardsdatascience.com/fine-tuning-pretrained-nlp-models-with-huggingfaces-trainer-6326a4456e7b

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification, RobertaForSequenceClassification, RobertaTokenizer
from transformers import EarlyStoppingCallback

# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item
        
        # item = {key: val[idx] for key, val in self.encodings.items()}
        # if self.labels:
        #     item["labels"] = self.labels[idx]
        # return item

    def __len__(self):
        return len(self.encodings["input_ids"])


# ----- 2. Fine-tune pretrained model -----#
# Define Trainer parameters
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='weighted')
    precision = precision_score(y_true=labels, y_pred=pred, average='weighted')
    f1 = f1_score(y_true=labels, y_pred=pred, average='weighted')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}
        
def train_with_data(model_name, X_train, X_val, y_train, y_val, save_model=True, model_output_dir=''):

  if len(model_output_dir)==0:
    model_output_dir = output_dir

  # tokenizer_path = model_name #"princeton-nlp/sup-simcse-roberta-large"

  # tokenizer = CURRENT_MODEL[1].from_pretrained(CURRENT_MODEL[2])
  # model = CURRENT_MODEL[0].from_pretrained(CURRENT_MODEL[2], num_labels=3)

  global MODELS

  CUR_MODEL = MODELS[model_name]

  tokenizer = CUR_MODEL['tokenizer'].from_pretrained(CUR_MODEL['path'])
  model = CUR_MODEL['classifier'].from_pretrained(CUR_MODEL['path'], num_labels=3)


  # tokenizer = RobertaTokenizer.from_pretrained(model_name)
  # model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=3)

  X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=CUR_MODEL['max_length'])
  X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=CUR_MODEL['max_length'])

  train_dataset = Dataset(X_train_tokenized, y_train)
  val_dataset = Dataset(X_val_tokenized, y_val)


  # Define Trainer
  args = TrainingArguments(
      output_dir=model_output_dir,
      evaluation_strategy="steps",
      logging_steps = CUR_MODEL['logging_steps'],
      per_device_train_batch_size=8,
      per_device_eval_batch_size=8,
      num_train_epochs=CUR_MODEL['num_train_epochs'],
      seed=0,
      load_best_model_at_end=True,
      learning_rate=CUR_MODEL['learning_rate'],
      
  )
  trainer = Trainer(
      model=model,
      args=args,
      train_dataset=train_dataset,
      eval_dataset=val_dataset,
      compute_metrics=compute_metrics,
      callbacks=[EarlyStoppingCallback(early_stopping_patience=CUR_MODEL['early_stopping_patience'])],
  )

  # Train pre-trained model
  trainer.train()

  if save_model == True:
    model.save_pretrained(model_output_dir)
  # trainer.evaluate()

  return trainer, model




In [17]:
def train_with_all_data(model_name='', file_path='', model_output_dir=''):
  # Read data
  data = get_ML_data("Train", file_path)
  X_train = list(data["text"])
  y_train = list(data["label"])


  data_val =  get_ML_data("Validate", file_path)
  X_val = list(data_val["text"])
  y_val = list(data_val["label"])

  # X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
  # print(X_train)

  if len(model_name)==0:
    # Define pretrained tokenizer and model
    # model_name = "roberta-large"
    # model_name = "ProsusAI/finbert"
    model_name = "princeton-nlp/sup-simcse-roberta-large"

  train_with_data(model_name, X_train, X_val, y_train, y_val, True, model_output_dir)

  df_val = pd.DataFrame({'text': X_val, 'label': y_val})
  #classify_with_data(read_clean_data(predict_file_path), "", "", True)

#train_with_all_data()

In [18]:
from sklearn.metrics import classification_report
import datetime

def write_report_internal(model, dataset_name, dataset_type, df, output_dir_report, duration, file_name, train_noise):
    target_names = ['positive', 'negative', 'neutral']
    global PREDICTION_RUN
    global MODELS
  
    # add model column and default value to df at the beginning of the dataframe
    df.insert(0, 'model', model)
    df.insert(1, 'dataset', dataset_name)
    df.insert(2, 'dataset_type', dataset_type)
    
    # get current date and time yyyy-mm-dd hh:mm
    now = datetime.datetime.now()
    dt_string = now.strftime("%Y-%m-%d %H:%M")
    df['time'] = dt_string
    df['duration'] = duration
    df['batch'] = PREDICTION_RUN
    df['settings'] = str(MODELS[model])
    df['data_file'] = file_name
    df['train_noise'] = train_noise
    


    # append the report to the existing csv file. Append the header only once
    with open(output_dir_report + '/classification_report.csv', 'a') as f:
        df.to_csv(f, header=f.tell() == 0, index=False)

def write_report(model, dataset_name, dataset_type, y_true, y_pred, output_dir_report, duration, file_name, train_noise):
    target_names = ['positive', 'negative', 'neutral']

    # # print the report in readable format
    print(classification_report(y_true, y_pred, target_names=target_names))
    
    # report = classification_report(y_true, y_pred, target_names=target_names, output_dict=True)
    report = classification_report(y_true, y_pred, output_dict=True)
    df = pd.DataFrame(report).transpose()
    # convert the index to a column 'class'
    df.reset_index(level=0, inplace=True)
    df.rename(columns={'index': 'class'}, inplace=True)
    write_report_internal(model, dataset_name, dataset_type, df, output_dir_report, duration, file_name, train_noise)

    # acc, pre, rec, f1 = compute_performance_opiner(y_true, y_pred)
    # df_opiner = pd.DataFrame({'class':['opiner'], 'support': [acc], 'precision': [pre], 'recall': [rec], 'f1': [f1]})
    # df_opiner.reset_index(level=0, inplace=True)
    # df.rename(columns={'index': 'class'}, inplace=True)
    # # df_opiner['class']='opiner'
    # write_report_internal(model, dataset_name, dataset_type, df_opiner, output_dir_report, duration)


def additional_classification(dataset_type, additional_testing_file_name, model, SEP, dataset, output_dir, duration, file_name, train_noise):
  combination_title = (model+SEP+dataset+SEP+dataset_type+SEP+train_noise).replace('/', '-')
  print_title(model, SEP, dataset, dataset_type)

  input_file_path = os.path.join(dataset_dir, additional_testing_file_name)
  prediction_output_file = os.path.join(prediction_output_dir, combination_title+SEP+additional_testing_file_name)
  accuracy, y_true, y_pred = classify(input_file_path, prediction_output_file, True, model_output_dir)
  write_report(model, dataset, dataset_type, y_true, y_pred, output_dir, duration, file_name, train_noise)

    

In [19]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel, RobertaForSequenceClassification, RobertaTokenizer
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from transformers import TrainingArguments, Trainer
import torch

import numpy as np
import pandas as pd

# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

  
def classify_with_data(test_data, output_file_path, generate_report, model_path ='', trainer = None):
  
  X_test = list(test_data["text"])
  
  # tokenizer_path = "princeton-nlp/sup-simcse-roberta-large"
  # tokenizer_path = 'roberta-large'
  #tokenizer_path = "ProsusAI/finbert"
  #tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
  
  # Since SimCSE also uses Roberta model, we are loading from 'roberta-large'.
  if model_path == 'cardiffnlp/twitter-roberta-base-sentiment-latest':
    print("*-"*100)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
  else:
    tokenizer = AutoTokenizer.from_pretrained('roberta-large')

  max_length=512

  X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=max_length)
  test_dataset = Dataset(X_test_tokenized)

  
  # # # Define test trainer
  if trainer != None:
    test_trainer = trainer
  else:
    if len(model_path) == 0:
      #model_path = "ProsusAI/finbert"
      #model_path = tokenizer_path
      model_path = output_dir

    print("MODEL PATH --------- "+model_path)

    model = AutoModelForSequenceClassification.from_pretrained(model_path) 

    test_trainer = Trainer(model)

  # # Make prediction
  raw_pred, _, _ = test_trainer.predict(test_dataset)
  
  # check if the model path is cardiffnlp/twitter-roberta-base-sentiment-latest
  if model_path == 'cardiffnlp/twitter-roberta-base-sentiment-latest':
        # move all the values in 3rd column to 1st column, and the values in the 1s column to 2nd column, 2nd column to 3rd column
    # since cardiffnlp/twitter-roberta-base-sentiment-latest provides output in neg/post/neutral order.
    raw_pred = np.roll(raw_pred, 1, axis=1)

  # Preprocess raw predictions
  data = test_data
  y_pred = np.argmax(raw_pred, axis=1)
  model_name = model_path.split('/')[-1] 
  data['sentiment-ml'+"-"+model_name] = y_pred
  prob = np.array(raw_pred)
  data['positive'+"-"+model_name] = prob[:,0]
  data['negative'+"-"+model_name] = prob[:,1]
  data['neutral'+"-"+model_name] = prob[:,2]
  # data['model'] = model_path.split('/')[-1] 


  if len(output_file_path) > 0:
    # file_path = output_file_path.replace(".csv", "_"+DATE+ str(PREDICTION_RUN) +".csv")
    
    # THIS IS TEMPORARY FOR CHECKING MULTIPLE MODEL PERFORMANCE
    file_path = output_file_path
    data.to_csv(file_path, index=False)

    print("Written prediction results to file: "+ file_path)

  if generate_report == True: #validation case only
    y_test = list(test_data["label"])
    from sklearn.metrics import classification_report
    print(classification_report(y_test, y_pred.tolist()))
    f1score = f1_score(y_test, y_pred, average='weighted')
    print(f1score)
    return f1score, y_test, y_pred.tolist()
    

  # ----- 3. Classify/Predict -----#
def classify(input_file_path, output_file_path, generate_report, model_path=''):
  test_data = get_ML_data("Test", input_file_path)
  file_path = output_file_path.replace(".csv", "_"+DATE+ str(PREDICTION_RUN) +"_relevant.csv")


  return classify_with_data(test_data, output_file_path, generate_report, model_path)


#PREDICTION_RUN += 1

In [31]:
import os
import time

# read data_files.csv file and go through each file and perform cross validation
df_data_files = pd.read_csv(os.path.join(dataset_dir, 'data_files.csv'))

# models = ["roberta-large"]
# models = ["princeton-nlp/sup-simcse-roberta-large"]
# models = ["roberta-large", "princeton-nlp/sup-simcse-roberta-large"]
# models = ["roberta", "simcse"]
# models = ["roberta"]
models = ["simcse"]
combine_classification_report = os.path.join(output_dir, "classification_report.csv")

# loop through each row in the dataframe
for index, row in df_data_files.iterrows():
  for model in models:
      # get the file path
      dataset = row['dataset']
      dataset_type = row['type']
      file_name = row['file']
      train = (row['train'] == 'yes')
      train_noise = row['train_noise']
      if train == False:
        continue

      SEP = '--'
      combination_title = (model+SEP+dataset+SEP+dataset_type+SEP+train_noise).replace('/', '-')
      print_title(model, SEP, dataset, dataset_type)

      input_file_path = os.path.join(dataset_dir, file_name)
      model_output_dir = os.path.join(model_output_dir_base, combination_title)
      prediction_output_file = os.path.join(prediction_output_dir, combination_title+SEP+file_name)


      print(input_file_path)
      print(model_output_dir)
      print(prediction_output_file)

      if not os.path.exists(model_output_dir):
        os.makedirs(model_output_dir)
      #PREDICTION_RUN += 1
  
      start_time = time.time()

      train_with_all_data(model, input_file_path, model_output_dir)
      # continue 

      end_time = time.time()
      duration = end_time - start_time

      print_title(model, SEP, dataset, dataset_type)
      f1score, y_true, y_pred = classify(input_file_path, prediction_output_file, True, model_output_dir)

      write_report(model, dataset, dataset_type, y_true, y_pred, output_dir, duration, file_name, train_noise)

      # additional_testing_file_name    = 'library-selection-factors.csv'
      # run_additional_testing = True

      if run_additional_testing:
        dataset_type = 'Factors'
        additional_testing_file_name    = 'library-selection-factors.csv'
        additional_classification(dataset_type, additional_testing_file_name, model, SEP, dataset, output_dir, duration, file_name, train_noise)

        dataset_type = 'Factors-Fixed-emotion'
        additional_testing_file_name    = 'library-selection-factors-emotion.csv'
        additional_classification(dataset_type, additional_testing_file_name, model, SEP, dataset, output_dir, duration, file_name, train_noise)
        
        dataset_type = 'Factors-Auto-emotion'
        additional_testing_file_name    = 'library-selection-factors-auto-emotion.csv'
        additional_classification(dataset_type, additional_testing_file_name, model, SEP, dataset, output_dir, duration, file_name, train_noise)


********************************************************************************
SIMCSE--4.3.BINLIN-SO--ML
********************************************************************************
/content/machine-learning/model-preparation/0.Combined-Dataset/65.4.3.StackOverflow-data-ML-auto-emotion-double-Train.csv
/content/machine-learning/model-output/model_files/simcse--4.3.BinLin-SO--ML--Auto-Emotion-D-T-All
/content/machine-learning/model-output/prediction_files/simcse--4.3.BinLin-SO--ML--Auto-Emotion-D-T-All--65.4.3.StackOverflow-data-ML-auto-emotion-double-Train.csv


Some weights of the model checkpoint at princeton-nlp/sup-simcse-roberta-large were not used when initializing RobertaForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at princeton-nlp/sup-simcse-roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this mode

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
100,0.564800,0.521031,0.827243,0.753090,0.827243,0.784275
200,0.388500,0.376401,0.893688,0.887934,0.893688,0.888676
300,0.259300,0.626111,0.877076,0.871704,0.877076,0.873533
400,0.152800,0.724849,0.887043,0.893853,0.887043,0.889679
500,0.156900,0.649864,0.890365,0.890022,0.890365,0.890096
600,0.093900,0.792493,0.877076,0.871405,0.877076,0.873096
700,0.023200,0.732964,0.893688,0.891841,0.893688,0.892522
800,0.000600,0.902329,0.880399,0.876847,0.880399,0.878123
900,0.006400,0.924186,0.880399,0.876658,0.880399,0.878039
1000,0.003400,0.920187,0.883721,0.880486,0.883721,0.881740


********************************************************************************
SIMCSE--4.3.BINLIN-SO--ML
********************************************************************************
MODEL PATH --------- /content/machine-learning/model-output/model_files/simcse--4.3.BinLin-SO--ML--Auto-Emotion-D-T-All


Written prediction results to file: /content/machine-learning/model-output/prediction_files/simcse--4.3.BinLin-SO--ML--Auto-Emotion-D-T-All--65.4.3.StackOverflow-data-ML-auto-emotion-double-Train.csv
              precision    recall  f1-score   support

           0       0.69      0.69      0.69        26
           1       0.72      0.72      0.72        36
           2       0.92      0.92      0.92       238

    accuracy                           0.88       300
   macro avg       0.78      0.78      0.78       300
weighted avg       0.88      0.88      0.88       300

0.88
              precision    recall  f1-score   support

    positive       0.69      0.69      0.69        26
    negative       0.72      0.72      0.72        36
     neutral       0.92      0.92      0.92       238

    accuracy                           0.88       300
   macro avg       0.78      0.78      0.78       300
weighted avg       0.88      0.88      0.88       300

*********************************

...

Written prediction results to file: /content/machine-learning/model-output/prediction_files/simcse--6.Opiner-StackOverflow--ML--Auto-Emotion-D-T-All--67.6.Opiner-ML-auto-emotion-double-Train.csv
              precision    recall  f1-score   support

           0       0.51      0.56      0.54       210
           1       0.63      0.54      0.58       168
           2       0.77      0.78      0.78       528

    accuracy                           0.68       906
   macro avg       0.64      0.63      0.63       906
weighted avg       0.69      0.68      0.68       906

0.684168948253111
              precision    recall  f1-score   support

    positive       0.51      0.56      0.54       210
    negative       0.63      0.54      0.58       168
     neutral       0.77      0.78      0.78       528

    accuracy                           0.68       906
   macro avg       0.64      0.63      0.63       906
weighted avg       0.69      0.68      0.68       906

*************************

Some weights of the model checkpoint at princeton-nlp/sup-simcse-roberta-large were not used when initializing RobertaForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at princeton-nlp/sup-simcse-roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this mode

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
100,0.522700,0.363481,0.877653,0.879435,0.877653,0.877655
200,0.408100,0.343885,0.902622,0.904501,0.902622,0.903047
300,0.300500,0.513331,0.892634,0.895950,0.892634,0.893020
400,0.302900,0.562039,0.881398,0.883542,0.881398,0.878129
500,0.295500,0.528109,0.880150,0.882660,0.880150,0.875512
600,0.275300,0.469686,0.901373,0.911732,0.901373,0.903536
700,0.264900,0.477591,0.902622,0.902353,0.902622,0.901814
800,0.082800,0.924314,0.861423,0.870240,0.861423,0.855684
900,0.124600,0.529661,0.923845,0.924199,0.923845,0.923811
1000,0.091100,0.578633,0.906367,0.905985,0.906367,0.905577


********************************************************************************
SIMCSE--7.1.BERT4SENTISE-STACKOVERFLOW--ML
********************************************************************************
MODEL PATH --------- /content/machine-learning/model-output/model_files/simcse--7.1.BERT4SentiSE-StackOverflow--ML--Auto-Emotion-D-T-All


Written prediction results to file: /content/machine-learning/model-output/prediction_files/simcse--7.1.BERT4SentiSE-StackOverflow--ML--Auto-Emotion-D-T-All--68.7.1.BERT4SentiSE.SO-ML-auto-emotion-double-Train.csv
              precision    recall  f1-score   support

           0       0.94      0.76      0.84       116
           1       0.97      0.80      0.88       224
           2       0.86      0.97      0.91       460

    accuracy                           0.90       800
   macro avg       0.92      0.85      0.88       800
weighted avg       0.90      0.90      0.89       800

0.8930917537746806
              precision    recall  f1-score   support

    positive       0.94      0.76      0.84       116
    negative       0.97      0.80      0.88       224
     neutral       0.86      0.97      0.91       460

    accuracy                           0.90       800
   macro avg       0.92      0.85      0.88       800
weighted avg       0.90      0.90      0.89       800



In [32]:
#cp /content/machine-learning/model-output/classification_report.csv /content/drive/MyDrive//machine-learning-output/
!cp /content/machine-learning/model-output/classification_report.csv /content/drive/MyDrive//machine-learning/model-output

In [33]:
#!cp -r               /content/machine-learning/model-output/* /content/drive/MyDrive//machine-learning/model-output/
!rsync -av --progress /content/machine-learning/model-output/* /content/drive/MyDrive//machine-learning/model-output/ --exclude model_files/

sending incremental file list
prediction_files/simcse--4.3.BinLin-SO--ML--Auto-Emotion-D-T-All--65.4.3.StackOverflow-data-ML-auto-emotion-double-Train.csv
         72,478 100%   37.87MB/s    0:00:00 (xfr#1, to-chk=35/83)
prediction_files/simcse--5.Senti4SD-GoldStandard-EmotionPolarity--ML--Auto-Emotion-D-T-All--66.05.Senti4SD_GoldStandard_EmotionPolarity-ML-auto-emotion-double-Train.csv
        454,545 100%   17.34MB/s    0:00:00 (xfr#2, to-chk=25/83)
prediction_files/simcse--6.Opiner-StackOverflow--ML--Auto-Emotion-D-T-All--67.6.Opiner-ML-auto-emotion-double-Train.csv
        300,753 100%    6.37MB/s    0:00:00 (xfr#3, to-chk=17/83)
prediction_files/simcse--7.1.BERT4SentiSE-StackOverflow--ML--Auto-Emotion-D-T-All--68.7.1.BERT4SentiSE.SO-ML-auto-emotion-double-Train.csv
        211,397 100%    3.15MB/s    0:00:00 (xfr#4, to-chk=6/83)

sent 1,047,895 bytes  received 105 bytes  2,096,000.00 bytes/sec
total size is 28,255,487  speedup is 26.96


In [31]:
import os

input_file_path = '/content/machine-learning/model-preparation/0.Combined-Dataset/stackoverflow_gson_cleaned.csv'
prediction_output_file = os.path.join(prediction_output_dir, 'stackoverflow_gson_test.csv')

input_file_path = '/content/machine-learning/model-output/prediction_files/stackoverflow_gson_test_2023_03_24_161620.csv'
prediction_output_file = input_file_path


models = [#'cardiffnlp/twitter-roberta-base-sentiment-latest', 
          #'/content/machine-learning/model-output/model_files/simcse--17.POME-StackOverflow--MLAuto-Emotion-D',
          # '/content/machine-learning/model-output/model_files/simcse--17.POME-StackOverflow--MLnone',
          # '/content/machine-learning/model-output/model_files/simcse--4.1.BinLin-AppReviews--MLAuto-Emotion-D',
          # '/content/machine-learning/model-output/model_files/simcse--4.1.BinLin-AppReviews--MLnone',
          # '/content/machine-learning/model-output/model_files/simcse--4.3.BinLin-SO--MLAuto-Emotion-D',
          # '/content/machine-learning/model-output/model_files/simcse--4.3.BinLin-SO--MLnone',
          # '/content/machine-learning/model-output/model_files/simcse--7.1.BERT4SentiSE-StackOverflow--MLAuto-Emotion-D',
          # '/content/machine-learning/model-output/model_files/simcse--7.1.BERT4SentiSE-StackOverflow--MLnone',
          '/content/machine-learning/model-output/model_files/simcse--4.3.BinLin-SO--ML--Auto-Emotion-D-T',
          '/content/machine-learning/model-output/model_files/simcse--5.Senti4SD-GoldStandard-EmotionPolarity--ML--Auto-Emotion-D-T',
          '/content/machine-learning/model-output/model_files/simcse--6.Opiner-StackOverflow--ML--Auto-Emotion-D-T',
          '/content/machine-learning/model-output/model_files/simcse--7.1.BERT4SentiSE-StackOverflow--ML--Auto-Emotion-D-T',
          ]

for model_output_dir in models:
  print(input_file_path)
  print(prediction_output_file)
  print(model_output_dir)

  classify(input_file_path, prediction_output_file, False, model_output_dir)


/content/machine-learning/model-output/prediction_files/stackoverflow_gson_test_2023_03_24_161620.csv
/content/machine-learning/model-output/prediction_files/stackoverflow_gson_test_2023_03_24_161620.csv
/content/machine-learning/model-output/model_files/simcse--4.3.BinLin-SO--ML--Auto-Emotion-D-T
MODEL PATH --------- /content/machine-learning/model-output/model_files/simcse--4.3.BinLin-SO--ML--Auto-Emotion-D-T


Written prediction results to file: /content/machine-learning/model-output/prediction_files/stackoverflow_gson_test_2023_03_24_161620.csv
/content/machine-learning/model-output/prediction_files/stackoverflow_gson_test_2023_03_24_161620.csv
/content/machine-learning/model-output/prediction_files/stackoverflow_gson_test_2023_03_24_161620.csv
/content/machine-learning/model-output/model_files/simcse--5.Senti4SD-GoldStandard-EmotionPolarity--ML--Auto-Emotion-D-T
MODEL PATH --------- /content/machine-learning/model-output/model_files/simcse--5.Senti4SD-GoldStandard-EmotionPolarity--ML--Auto-Emotion-D-T


Written prediction results to file: /content/machine-learning/model-output/prediction_files/stackoverflow_gson_test_2023_03_24_161620.csv
/content/machine-learning/model-output/prediction_files/stackoverflow_gson_test_2023_03_24_161620.csv
/content/machine-learning/model-output/prediction_files/stackoverflow_gson_test_2023_03_24_161620.csv
/content/machine-learning/model-output/model_files/simcse--6.Opiner-StackOverflow--ML--Auto-Emotion-D-T
MODEL PATH --------- /content/machine-learning/model-output/model_files/simcse--6.Opiner-StackOverflow--ML--Auto-Emotion-D-T


Written prediction results to file: /content/machine-learning/model-output/prediction_files/stackoverflow_gson_test_2023_03_24_161620.csv
/content/machine-learning/model-output/prediction_files/stackoverflow_gson_test_2023_03_24_161620.csv
/content/machine-learning/model-output/prediction_files/stackoverflow_gson_test_2023_03_24_161620.csv
/content/machine-learning/model-output/model_files/simcse--7.1.BERT4SentiSE-StackOverflow--ML--Auto-Emotion-D-T
MODEL PATH --------- /content/machine-learning/model-output/model_files/simcse--7.1.BERT4SentiSE-StackOverflow--ML--Auto-Emotion-D-T


Written prediction results to file: /content/machine-learning/model-output/prediction_files/stackoverflow_gson_test_2023_03_24_161620.csv
